In [17]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim_models


In [18]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [19]:
stopwords = stopwords.words("english")

In [20]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [21]:
data = load_data("C:/Users/yigit/Desktop/ushmm_dn.json")["texts"]

print (data[0][0:90]) 

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928. 


In [22]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

name be bear small town call be bear be very hard work child father mother have small mill


In [23]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['name', 'be', 'bear', 'small', 'town', 'call', 'be', 'bear', 'be', 'very', 'hard', 'work', 'child', 'father', 'mother', 'have', 'small', 'mill', 'flour', 'buckwheat']


In [24]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 11), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 2), (8, 3), (9, 1), (10, 12), (11, 1), (12, 8), (13, 1), (14, 2), (15, 1), (16, 3), (17, 2), (18, 1), (19, 2)]
able


In [29]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [30]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.034841  0.007122       1        1  46.012312
3      0.045297 -0.060109       2        1  27.074597
1     -0.015872  0.091229       3        1  18.780290
2     -0.064265 -0.038241       4        1   8.132802, topic_info=        Term           Freq          Total Category  logprob  loglift
78        be  137416.000000  137416.000000  Default  30.0000  30.0000
345       go   23651.000000   23651.000000  Default  29.0000  29.0000
2283  ghetto    2551.000000    2551.000000  Default  28.0000  28.0000
594   people   12479.000000   12479.000000  Default  27.0000  27.0000
169     come   13988.000000   13988.000000  Default  26.0000  26.0000
...      ...            ...            ...      ...      ...      ...
294   family     375.926179    2656.299468   Topic4  -5.5225   0.5540
92       big     359.892605    2291.147733   Topic4  -5.5661   0.6583
809     then     420.647721    7921.443996   Topic4  -5.4101  -0.4263
766    start     379.350135    3636.407266   Topic4  -5.5135   0.2490
810    there     429.156724   11376.088426   Topic4  -5.3901  -0.7682

[374 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
918       1  0.009814   access
918       2  0.024535   access
918       3  0.966689   access
923       1  0.100432   action
923       2  0.047573   action
...     ...       ...      ...
908       4  0.042655     year
2543      1  0.041262  zionist
2543      2  0.178803  zionist
2543      3  0.295713  zionist
2543      4  0.488271  zionist

[704 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 3])